# Day 13: Mine Cart Madness
[link](https://adventofcode.com/2018/day/13)

## Parsing inputs

In [1]:
test_in = [
'/->-\        ',
'|   |  /----\\',
'| /-+--+-\  |',
'| | |  | v  |',
'\-+-/  \-+--/',
'  \------/   ']

print('\n'.join(test_in))
print(set(len(s) for s in test_in))

/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
{13}


In [2]:
with open('input.txt') as file:
  puzzle_in =[line.strip('\n') for line in file]

print(set(len(s) for s in puzzle_in))

{150}


## Part 1

Find `X,Y` coordinates of the first crash

In [3]:
def turns_gen():
  while True:
    for turn in 'lsrs':
      yield turn

class Cart:
  def __init__(self, direction):
    self.direction = direction
    self.turns_gen_ = turns_gen()

  def turn_dir(self):
    return next(self.turns_gen_)

  def __repr__(self):
    return self.direction

In [4]:
next_position = {'>': (1,0),'<': (-1,0),'^': (0,-1),'v': (0,1)}

class Mine:
  carts = {}
  tracks = []

  def __init__(self, input_str):
    for y,line in enumerate(input_str):
      track_line = []
      for x,char in enumerate(line):
        if char in '<v>^':
          self.carts[(y,x)] = Cart(char)
          track_line.append('|' if char in '^v' else '-')
        else:
          track_line.append(char)
      self.tracks.append(track_line)

  def __repr__(self):
    return '\n'.join(''.join(char if (y,x) not in self.carts else self.carts[(y,x)].direction for x, char in enumerate(line)) for y,line in enumerate(self.tracks))

  def tick(self):
    for (y,x) in sorted(self.carts.keys()):
      cart = self.carts.pop((y,x))
      curr_track = self.tracks[y][x]
      if curr_track in '-|':
        position_increment = next_position[cart.direction]
        new_x = x+position_increment[0]
        new_y = y+position_increment[1]
        self.carts[(new_y,new_x)] = cart
      print(x,y,cart, curr_track)

test_mine = Mine(test_in)
print(test_mine.carts)

assert str(test_mine)=='\n'.join(test_in)

print(test_mine)

test_mine.tick()
print(test_mine)

{(0, 2): >, (3, 9): v}
/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
2 0 > -
9 3 v |
/-->\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-v--/
  \------/   


In [5]:
test_mine = Mine(test_in)
test_mine

/->>\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-v--/
  \------/   
/---\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

In [6]:
tt = {(2,0),(1,3),(1,1)}
sorted(tt)

[(1, 1), (1, 3), (2, 0)]